In [1]:
# mount drive and add path to root folder so custom_GANs python package is available
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.insert(0,'/content/drive/My Drive/generative_models')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [3]:
from pathlib import Path
import os

import tensorflow as tf
import keras
import numpy as np

import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

from src.custom_GANs.simple_GANs import ConditionalGAN, ModelMonitor, CheckpointCleanupCallback, GANCheckpoint, ConditionalGenerator, ConditionalDiscriminator


In [4]:
batch_size = 128
epochs = 100
num_channels = 1
num_example_images = 3
num_classes = 10
image_size = 28
latent_dim = 128
output_dim = (image_size, image_size, 1)


In [5]:
def scale_images(images, labels):
    image = tf.cast(images, tf.float32)
    # Scale the image
    image = (image - 127.5) / 127.5
    return image, labels

In [6]:
# We'll use all the available examples from both the training and test
# sets.
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_labels = np.concatenate([y_train, y_test])
all_labels = all_labels.reshape(-1, 1)
# Scale the pixel values to [0, 1] range, add a channel dimension to
# the images, and one-hot encode the labels.
all_digits = np.reshape(all_digits, (-1, 28, 28, 1))

# Create tf.data.Dataset.
dataset = tf.data.Dataset.from_tensor_slices((all_digits, all_labels))
dataset = dataset.map(scale_images)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {all_digits.shape}")
print(f"Shape of training labels: {all_labels.shape}")

Shape of training images: (70000, 28, 28, 1)
Shape of training labels: (70000, 1)


In [7]:
example_images_dir =  Path('/content/drive/My Drive/generative_models').resolve() / 'examples/images/CGAN'
example_images_dir.mkdir(parents=True, exist_ok=True)
print('Example images save dir: ', example_images_dir)

checkpoint_dir =  Path('/content/drive/My Drive/generative_models').resolve() / 'examples/checkpoints/CGAN'
checkpoint_dir.mkdir(parents=True, exist_ok=True)
print('Checkpoint save dir: ', checkpoint_dir)

Example images save dir:  /content/drive/My Drive/generative_models/examples/images/CGAN
Checkpoint save dir:  /content/drive/My Drive/generative_models/examples/checkpoints/CGAN


In [8]:
checkpoint_callback = GANCheckpoint(checkpoint_dir=checkpoint_dir)

noise_generator = lambda: tf.random.uniform((num_example_images, latent_dim,1))
label_generator = lambda: tf.random.uniform((num_example_images, 1), minval=0, maxval=num_classes, dtype=tf.int32 )

generator_input_generators = [noise_generator, label_generator]
checkpoint_model_monitor = ModelMonitor(save_path = example_images_dir, input_generators = generator_input_generators)


In [9]:
[func() for func in generator_input_generators]

[<tf.Tensor: shape=(3, 128, 1), dtype=float32, numpy=
 array([[[1.38393760e-01],
         [9.03420925e-01],
         [4.60731983e-03],
         [1.41658306e-01],
         [5.42865157e-01],
         [1.36314631e-01],
         [2.45056748e-01],
         [8.75133514e-01],
         [1.83089137e-01],
         [7.39388943e-01],
         [6.66017532e-01],
         [6.48572326e-01],
         [8.63163590e-01],
         [9.42645073e-01],
         [7.93363690e-01],
         [7.12498426e-01],
         [9.44967628e-01],
         [2.87255049e-02],
         [9.70601439e-01],
         [8.58486533e-01],
         [9.82882977e-02],
         [6.16692305e-02],
         [3.00984383e-02],
         [8.16752553e-01],
         [5.23593068e-01],
         [6.45159602e-01],
         [2.70596981e-01],
         [7.44751096e-01],
         [9.64350581e-01],
         [4.00631785e-01],
         [3.58242273e-01],
         [6.75010324e-01],
         [8.83495808e-03],
         [4.91514921e-01],
         [9.67377424e-02],
 

In [10]:
cgan = ConditionalGAN(latent_dim=latent_dim, output_dim=output_dim, nr_classes=num_classes)
cgan.compile()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
cgan.fit(dataset, epochs = epochs, batch_size = batch_size, callbacks = [checkpoint_model_monitor, checkpoint_callback, CheckpointCleanupCallback(checkpoint_dir)])

Epoch 1/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - d_loss: 0.7213 - g_loss: 0.6455ModelMonitor: Saving image to  /content/drive/My Drive/generative_models/examples/images/CGAN
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 60s 44ms/step - d_loss: 0.7213 - g_loss: 0.6457
Epoch 2/100
1093/1094 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - d_loss: 0.6815 - g_loss: 0.7690ModelMonitor: Saving image to  /content/drive/My Drive/generative_models/examples/images/CGAN
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 68s 38ms/step - d_loss: 0.6815 - g_loss: 0.7690
Epoch 3/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - d_loss: 0.6798 - g_loss: 0.7740ModelMonitor: Saving image to  /content/drive/My Drive/generative_models/examples/images/CGAN
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 82s 38ms/step - d_loss: 0.6798 - g_loss: 0.7740
Epoch 4/100
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - d_loss: 0.6522 - g_loss: 0.8043ModelMonitor: Saving image to  /content/drive/My Drive/generative_models/examples/images/CGAN
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 